# Model Selection - 1
- **k-fold cross-validation** using ```scikit-learn wrapper```
- **grid search** using ```scikit-learn wrapper```
- **random search** using ```scikit-learn wrapper```

## **k-fold cross validation**
- One of the most common method for model validation and selection
    - Partition training dataset into ```k``` subsets and choose one of partitions as **validation set** and other remaining subset as **training set**
    - Then, train model using **training set** and validate using **validation set**
    - Average validation results of ```k``` rounds of partitions and training/validation
    - Compare the results
    
<img src="https://upload.wikimedia.org/wikipedia/commons/1/1c/K-fold_cross_validation_EN.jpg" style="width: 500px"/>

## **Grid search & Random search**
- **Grid search** and **random search** are two methods for **hyperparameter tuning**
    - **Grid search** tries all possible combinations of hyperparameter values specified
    - **Random search** implements *randomized search over parameters*, in which each trial is a sample from possible hyperparameter distributions
    
<img src="https://cdn-images-1.medium.com/max/1600/1*ZTlQm_WRcrNqL-nLnx6GJA.png" style="width: 500px"/>

### Load dataset
- ```imdb``` dataset

In [1]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

In [2]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = 5000)

17465344/17464789 [==============================] - 0s 0us/step


In [3]:
# printing out maximum & minimun length of sentences
print(len(max(X_train, key = len)))
print(len(max(X_test, key = len)))
print(len(min(X_train, key = len)))
print(len(min(X_test, key = len)))

2494
2315
11
7


In [4]:
X_train = sequence.pad_sequences(X_train, maxlen = 500)
X_test = sequence.pad_sequences(X_test, maxlen = 500)

In [5]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000, 500)
(25000, 500)
(25000,)
(25000,)


- When using ```Sequential``` model API, ```scikit-learn wrapper``` can be used
- Define function to create model, and pass on to ```KerasClassifier```

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, cross_val_score

In [22]:
def imdb_model(embed_dim = 100, lstm = True, lr = 0.01):
    model = Sequential()
    model.add(Embedding(5000, embed_dim))
    if lstm:
        model.add(LSTM(15))
    else:
        model.add(GRU(15))
            
    model.add(Dense(50))
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    adam = optimizers.Adam(lr = lr)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

In [23]:
model = KerasClassifier(build_fn = imdb_model, epochs = 3, batch_size = 128, verbose = 1)

### *k*-fold cross-validation using ```scikit-learn wrapper```

In [24]:
# set k = 3
results = cross_val_score(model, X_train, y_train, cv = 3)

Epoch 1/3
131/131 [==============================] - 39s 294ms/step - loss: 0.5311 - accuracy: 0.7419
Epoch 2/3
131/131 [==============================] - 39s 301ms/step - loss: 0.4161 - accuracy: 0.8217
Epoch 3/3
66/66 [==============================] - 5s 80ms/step - loss: 0.3227 - accuracy: 0.8708
Epoch 1/3
131/131 [==============================] - 39s 300ms/step - loss: 0.5366 - accuracy: 0.7292
Epoch 2/3
131/131 [==============================] - 40s 302ms/step - loss: 0.4613 - accuracy: 0.7845
Epoch 3/3
66/66 [==============================] - 5s 78ms/step - loss: 0.2930 - accuracy: 0.8862
Epoch 1/3
131/131 [==============================] - 44s 338ms/step - loss: 0.5585 - accuracy: 0.7136
Epoch 2/3
131/131 [==============================] - 39s 298ms/step - loss: 0.4365 - accuracy: 0.8040
Epoch 3/3
66/66 [==============================] - 5s 75ms/step - loss: 0.3453 - accuracy: 0.8577


In [25]:
# print out results
# in most cases, average accuracy and standard deviation are meaningful metrics
print('Cross validation Results: ')
for i in range(len(results)):
    print('{}th round accuracy: {}'.format(i+1, results[i]))
print('Average accuracy: ', results.mean())
print('Standard deviation: ', results.std())

Cross validation Results: 
1th round accuracy: 0.8707703351974487
2th round accuracy: 0.8862354755401611
3th round accuracy: 0.8576743006706238
Average accuracy:  0.8715600371360779
Standard deviation:  0.011673414220526314


### grid search  using ```scikit-learn wrapper```

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
# first define hyperparameter grid
embed_dim = [100, 300]
lstm = [True, False]
lr = [0.001, 0.01]
batch_size = [64, 128, 256]

In [28]:
hyperparam_grid = {'embed_dim': embed_dim, 'lstm': lstm, 'lr': lr, 'batch_size': batch_size}

In [29]:
model = KerasClassifier(build_fn = imdb_model, epochs = 5, verbose = 1)
clf = GridSearchCV(estimator = model, param_grid = hyperparam_grid)

In [ ]:
grid_result = clf.fit(X_train, y_train)

In [ ]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

In [ ]:
# displaying best results & parameter settings
max_idx = np.argmax(means)
print('Best test accuracy: ', means[max_idx])
print('Standard Deviation of Accuracies: ', stds[max_idx])
print('Parameter Setting: ', params[max_idx])

### random search using ```scikit-learn wrapper```

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import numpy as np

In [ ]:
embed_dim = randint(100, 300)
lstm = [True, False]
lr = uniform(0.001, 0.1)
batch_size = randint(64, 256)

In [ ]:
hyperparam_dist = {'embed_dim': embed_dim, 'lstm': lstm, 'lr': lr, 'batch_size': batch_size}

In [ ]:
model = KerasClassifier(build_fn = imdb_model, epochs = 5, verbose = 1)
clf = RandomizedSearchCV(estimator = model, param_distributions = hyperparam_dist, n_iter = 20)

In [ ]:
random_result = clf.fit(X_train, y_train)

In [ ]:
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
params = clf.cv_results_['params']

In [ ]:
# displaying best results & parameter settings
max_idx = np.argmax(means)
print('Best test accuracy: ', means[max_idx])
print('Standard Deviation of Accuracies: ', stds[max_idx])
print('Parameter Setting: ', params[max_idx])

Best test accuracy:  0.861520008204
Standard Deviation of Accuracies:  0.00517948234862
Parameter Setting:  {'batch_size': 153, 'embed_dim': 121, 'lr': 0.0052380206595373227, 'lstm': True}
